## 1. Introduction


### 1.1. Description 

Dans ce projet, nous allons travailler sur un système de **génération augmentée de récupération**, ou **RAG** (*Retrieval Augmented Generation* en anglais), qui combine des modèles de langue génératifs et des techniques de recherche d'information.  

À partir d'une question donnée, nous devrons identifier les passages les plus pertinents à l'aide d'un modèle de recherche, puis générer une réponse en utilisant ces passages de texte. Enfin, nous explorerons différentes approches pour améliorer les performances de notre modèle RAG.  

### Description des données

Les passages donnés sont des textes en anglais de quelques phrases sur divers sujets. Par exemple, les mathématiques, la physique, la chimie, la biologie l'informatique, la musique et la psychologie. En général, les textes abordent des sujets très spécifiques reliés à un domaine. Par exemple, il pourrait y avoir 1000 textes parlant d'informatique et 100 qui abordent, à leur manière, la notion de mémoire. Parmi ces 100 passages, 5 pourraient aborder la mémoire dans un processeur spécifique.

En ce qui concerne les questions, elles sont basées sur les sujets de certains textes précis et on peut y répondre à l'aide d'au moins un des textes. Dans certains cas, plusieurs textes sont nécessaires pour répondre à la question. Par exemple, plusieurs textes pourraient aborder le processeur Intel i7-13700k, mais la question pourrait nécessiter l'information de tous ces textes pour trouver la réponse.
Les questions ont été créées pour faire en sorte que les réponses soient assez courtes. Il peut s'agir d'un simple nombre ou de quelques mots (< 30 mots).


Vous trouverez ~ 13 000 passages dans le corpus pour ~ 1 700 paires de questions / réponses dans le jeu d'entraînement et ~ 500 dans le jeu de validation. Vous devez prédire 500 réponses aux questions du jeu de test.

### Exemple

Par exemple, pour la question

> What type of bonds are used to form branches in glycogen?

Avec l'approche RAG, plutôt que de générer la réponse directement, on va d'abord chercher dans un corpus de passages. Dans cet exemple, un des passages pertinents est :

> **Glycogen Structure and Function** : Glycogen is a molecular polymer of glucose used for energy storage. It is composed of linear chains of glucose molecules linked by α-1,4-glycosidic bonds, with branches formed off the chain via α-1,6-glycosidic bonds. The branches provide additional ""free ends"" for linear chains, allowing for faster glucose release.

Puis on génère une réponse conditionnée par les passages pertinents :

> α-1,6-glycosidic bonds

### Motivation

Le RAG est une approche très populaire en ce moment. Elle permet notamment d'avoir des sources qui supportent les réponses générées, ce qui peut être utile pour la vérification de faits ou pour contrôler les hallucinations. De plus, elle permet d'intégrer des connaissances externes et récentes dans les modèles de génération sans avoir à les ré-entraîner.

### Objectif du projet  

Dans ce projet, nous allons implémenter plusieurs systèmes de question-réponse. Tout d'abord, nous testerons une approche de *prompting* qui ne s'appuie sur aucun passage pour aider le modèle de génération. Ensuite, nous mettrons en place un système figé utilisant un modèle de plongements statique pour retrouver les *k* passages les plus pertinents, qui seront ensuite fournis à un modèle de génération figé.  

Enfin, nous développerons une nouvelle méthode en nous basant sur les dernières avancées de l'état de l'art afin d'optimiser les performances et d'atteindre des meilleurs résultats.

### Jeux de données

Nous avons 4 fichiers à notre disposition :
- 'rag_texts.csv' : le corpus de passages extraits de Wikipédia.
  - **id** : l'identifiant unique du passage.
  - **text** : le texte du passage.
- 'rag_questions_train.csv', 'rag_questions_val.csv' et 'rag_questions_test.csv' : les questions d'entraînement, de validation et de test.
  - **id** : l'identifiant unique de la question.
  - **question** : La question.
  - **text_id** (sauf pour le fichier de test) : la liste des identifiants des passages pertinents du corpus 'rag_texts.csv' pour la question.
  - **answer** (sauf pour le fichier de test) : la réponse à la question.

### Sortie :  

En sortie, notre modèle produira un fichier CSV contenant deux colonnes :  
- **id** : l'identifiant de la question dans le jeu de données de test.  
- **answer** : la réponse générée pour cette question.  

Afin d'évaluer les performances de notre modèle sur l'ensemble de validation, nous utiliserons la métrique **BLEU**, qui permet de mesurer la similarité entre les réponses générées et les réponses de référence en comparant les n-grammes communs. Cette métrique est couramment utilisée pour évaluer la qualité des modèles de génération de texte, en particulier en traduction automatique et en question-réponse.

### Note:

- **Modèle génératif** : afin de mieux comparer les différentes approches, nous avons choisi d'utiliser exclusivement le modèle [microsoft/Phi-3-mini-128k-instruct](https://huggingface.co/microsoft/Phi-3-mini-128k-instruct), disponible sur Hugging Face. Cette contrainte garantit que les performances ne dépendent pas uniquement de la puissance du modèle de génération.  
- **Modèles de plongements à utiliser** : [BAAI/bge-small-en-v1.5](https://huggingface.co/BAAI/bge-small-en-v1.5).

### 1.2. Description des données et métriques d’évaluation


In [1]:
!pip install datasets accelerate

In [2]:
!pip install faiss-gpu

In [3]:
# Import des librairies

import os
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from typing import Optional
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from nltk.translate.bleu_score import sentence_bleu
import nltk
import faiss

/teamspace/studios/this_studio/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Si vous stockez vos données sur Google Drive

# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
# root_path = '/content/drive/MyDrive/'
# data_path = root_path + 'dataTp4/'

data_path = 'data/'

## 2. Analyse exploratoire et modèle de génération simple 

Pour commencer ce projet et établir des modèles de base (*baselines*), nous allons utiliser un modèle de génération simple pour répondre aux questions. Nous travaillerons avec le modèle **'microsoft/Phi-3-mini-128k-instruct'**, disponible sur Hugging Face.  

Dans cette première étape, nous générerons des réponses aux questions de l’ensemble de validation (*questions_val.csv*) sans utiliser de passages pour aider le modèle.

### 2.1. Chargement des données 

#### 2.1.1 Taille des données 

Affichons la taille de tous les jeux de données et quelques questions de l'ensemble d'entraînement

In [6]:
# TODO
train = pd.read_csv(data_path + 'questions_train.csv')
val = pd.read_csv(data_path + 'questions_val.csv')
test = pd.read_csv(data_path + 'questions_test.csv')
text = pd.read_csv(data_path + 'texts.csv')

Conversion des indices en liste de integer

In [7]:
train['text_ids'] = train["text_ids"].apply(lambda x: [int(i) for i in x.replace("[", "").replace("]", "").split(" ") if i.isdigit()])
val['text_ids'] = val["text_ids"].apply(lambda x: [int(i) for i in x.replace("[", "").replace("]", "").split(" ") if i.isdigit()])

In [8]:
print(f"Taille de l'ensemble d'entraînement: {len(train)}")
print(f"Taille de l'ensemble de validation: {len(val)}")
print(f"Taille de l'ensemble de test: {len(test)}")

Taille de l'ensemble d'entraînement: 1747
Taille de l'ensemble de validation: 500
Taille de l'ensemble de test: 500


In [9]:
train["question"].head()
# TODO

0    What type of insects are vectors for the Haemo...
1    What is the basis of the security of the BBS a...
2    What is the purpose of the catalase test in ba...
3    What type of cells clear small particles in ve...
4    What information is needed to decrypt a messag...
Name: question, dtype: object

#### 2.1.2 Analyse exploratoire 

a) Sur l'ensemble d'entraînement, affichons :
- Le nombre moyen de mots dans une question
- Le nombre moyen de mots dans une réponse
- Le nombre moyen de passages nécessaires pour répondre à une question
- Le nombre minimal de passages nécessaires pour répondre à une question
- Le nombre maximal de passages nécessaires pour répondre à une question


In [11]:
# TODO
mean_word_q = train["question"].apply(lambda x: len(x.split())).mean()
print(f"Le nombre moyen de mots dans une question: {mean_word_q}")

mean_word_a = train["answer"].apply(lambda x: len(x.split())).mean()
print(f"Le nombre moyen de mots dans une réponse: {mean_word_a}")

mean_passage = train['text_ids'].apply(lambda x: len(x)).mean()
print(f"Le nombre moyen de passages nécessaires pour répondre à une question: {mean_passage}")

min_passage = train['text_ids'].apply(lambda x: len(x)).min()
print(f"Le nombre minimal de passages nécessaires pour répondre à une question: {min_passage}")

max_passage = train['text_ids'].apply(lambda x: len(x)).max()
print(f"Le nombre maximal de passages nécessaires pour répondre à une question: {max_passage}")

# END TODO

Le nombre moyen de mots dans une question: 10.37263880938752
Le nombre moyen de mots dans une réponse: 6.084144247281054
Le nombre moyen de passages nécessaires pour répondre à une question: 2.839152833428735
Le nombre minimal de passages nécessaires pour répondre à une question: 2
Le nombre maximal de passages nécessaires pour répondre à une question: 4


b) De plus, affichons des histogrammes décrivant la distribution du nombre de mots des questions, des réponses et des textes. Utilisez des bacs (bins) de 50. 

In [ ]:
# TODO
plt.hist(train["question"].apply(lambda x: len(x.split())), bins=50)
plt.title("Distribution du nombre de mots des questions")
plt.xlabel("Nombre de mots")
plt.ylabel("Fréquence")
plt.show()


In [ ]:
plt.hist(train["answer"].apply(lambda x: len(x.split())), bins=50)
plt.title("Distribution du nombre de mots des réponses")
plt.xlabel("Nombre de mots")
plt.ylabel("Fréquence")
plt.show()

In [ ]:
plt.hist(text["text"].apply(lambda x: len(x.split())), bins=50)
plt.title("Distribution du nombre de mots des passages ")
plt.xlabel("Nombre de mots")
plt.ylabel("Fréquence")
plt.show()
#END TODO

COMMENTAIRE:

- Les questions sont en moyenne plus longues que les réponses

- Les réponses sont très courtes

- Les passages font en moyenne 40 mots

### 2.2. Chargement du modèle et génération des réponses 

Nous allons maintenant évaluer la performance d'un modèle de langue à répondre aux questions de l'ensemble de validation. Chargeons d'abord le modèle **'microsoft/Phi-3-mini-128k-instruct'** avec la librairie `transformers` de huggingface et générons les réponses du modèle à partir de la question. 

In [12]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 16

READER_MODEL_NAME = "microsoft/Phi-3-mini-128k-instruct"

In [13]:
# TODO

model_reader = AutoModelForCausalLM.from_pretrained(
    READER_MODEL_NAME,
)
tokenizer_reader = AutoTokenizer.from_pretrained(READER_MODEL_NAME)

READER_LLM = pipeline(
    model=model_reader,
    tokenizer=tokenizer_reader,
    task="text-generation",
    do_sample=True,
    temperature=0.7,
    repetition_penalty=1.1,
    return_full_text=False,
    device= 0 if DEVICE == "cuda" else -1,
    max_new_tokens=32,
)
# END TODO

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.28s/it]


### 2.3. Évaluation des réponses 

Nous allons créer la fonction `evaluate_bleu` et utiliser la métrique BLEU pour évaluer les réponses générées par rapport aux réponses attendues sur le jeu de validation. Nous évaluerons notre modèle avec les métriques **BLEU-1** et **BLEU-2** :  

- **BLEU-1** prend en compte uniquement les unigrammes pour évaluer la qualité des réponses.  
- **BLEU-2** intègre également les bigrammes pour une évaluation plus fine.  



In [14]:
def evaluate_bleu(df_true: pd.DataFrame, df_pred: pd.DataFrame, bleu_type: int):
    """
    Évalue les réponses générées en utilisant la métrique BLEU.

    Paramètres:
    df_true (pd.DataFrame): DataFrame contenant les vraies réponses avec des colonnes 'id' et 'answer'.
    df_pred (pd.DataFrame): DataFrame contenant les réponses prédites avec des colonnes 'id' et 'answer'.
    bleu_type (int): Nombre (soit 1 ou 2) correspondant aux n-grammes considérés pour la métrique (bleu_type = 1 : BLEU-1, bleu_type = 2 : BLEU-2)

    Retourne:
    float: Score BLEU moyen sur toutes les entrées.
    """
    # TODO
    scores = []
    for i in range(len(df_true)):
        true_answer = df_true.iloc[i]['answer']
        pred_answer = df_pred.iloc[i]['answer']
        if bleu_type == 1:
            scores.append(sentence_bleu([true_answer.split()], pred_answer.split(),weights=(1, 0, 0, 0)))
        elif bleu_type == 2:
            scores.append(sentence_bleu([true_answer.split()], pred_answer.split(),weights=(0, 1, 0, 0)))
        else:
            raise ValueError("bleu_type 1 or 2")
    return np.mean(scores)

    # END TODO

In [16]:
# TODO

questions = val["question"].tolist()

predictions = READER_LLM(questions, batch_size=1)

pred = [output[0]["generated_text"] for output in predictions]

torch.cuda.empty_cache()

df_pred = pd.DataFrame({"id": val["id"], "answer": pred})
df_true = val[["id", "answer"]]
# TODO

In [17]:
bleu_1 = evaluate_bleu(df_true, df_pred, 1)
bleu_2 = evaluate_bleu(df_true, df_pred, 2)

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [18]:
print(f"BLEU-1: {bleu_1}")
print(f"BLEU-2: {bleu_2}")

BLEU-1: 0.06517458063112228
BLEU-2: 0.012476048968439521


BLEU-1: 0.06517458063112228
BLEU-2: 0.012476048968439521

### 2.4. Commentaire 




Performance obtenue:

- BLEU-1: 0.06662334044572885

- BLEU-2: 0.015467704378992813

-> Les perfomances sont très faibles. Le modèle n'a pas été entrainé sur ce sujet, ne peux donc répondre sans contexte.


## 3. Approche RAG figé 
.lightning_studio
Dans cette partie, nous allons implémenter une approche simple de **RAG figé**. Ce modèle est qualifié de figé car aucun ré-entraînement n’est effectué. Nous mettrons en place deux variantes : avec ou sans l'utilisation d'un algorithme d'indexation.  

Dans cette approche, nous commencerons par retrouver les passages pertinents pour chaque question à l'aide d'un modèle de plongements basé sur Transformers, puis nous générerons une réponse conditionnée par ces passages.  

Pour cela, nous utiliserons :  
- **[BAAI/bge-small-en-v1.5](https://huggingface.co/BAAI/bge-small-en-v1.5)** comme modèle de plongements (*embedding model*) afin d'obtenir les vecteurs des questions et des passages.  
- **[microsoft/Phi-3-mini-128k-instruct](https://huggingface.co/microsoft/Phi-3-mini-128k-instruct)** comme modèle de génération (*generative model*) pour produire les réponses.

### 3.1 RAG figé sans indexation 

In [15]:
EMBED_MODEL_NAME = "BAAI/bge-small-en-v1.5"

#### 3.1.1 Passages pertinents 

Pour retrouver les passages pertinents, nous avons encodé les questions et les passages avec le modèle de plongements. Puis, nous avons calculé la similarité cosinus entre les questions et les passages pour retrouver les $k$ passages les plus pertinents pour chaque question.

In [16]:
# TODO
tokenizer = AutoTokenizer.from_pretrained(EMBED_MODEL_NAME)
model = AutoModel.from_pretrained(EMBED_MODEL_NAME)
# END TODO

In [17]:
def encode_sequences(sequences: list, tokenizer, model, device = DEVICE, batch_size = BATCH_SIZE):
    """
    Encode les textes en utilisant le modèle passé en paramètre pour générer les plongements des textes

    Paramètres:
    sequences    : Liste de séquence à transformer en plongements
    tokenizer   : Segmenteur du modèle de plongements
    model       : Modèle de plongements
    device      : Machine sur laquelle les opérations doivent être effectuées
    batch_size  : Taille des lots lors de la génération des traitements
    """
    # TODO

    model.to(device)
    model.eval()

    embeddings = []
    with torch.no_grad():
        for i in range(0, len(sequences), batch_size):

            batch_sequences = sequences[i:i + batch_size]

            # Tokenizer
            tokens_sentences = tokenizer(batch_sequences, padding=True, truncation=True, return_tensors="pt").to(device)
            #Embedding
            embed_sentences = model(**tokens_sentences)

            embeddings.extend(embed_sentences[0][:,0])

    embeddings = torch.stack(embeddings)
    return embeddings
    # END TODO

In [18]:
# TODO

encoded_passages = encode_sequences(text["text"].to_list(), tokenizer, model)
# END TODO

#### 3.1.2 Évaluation des passages retrouvés avec Recall@k et precision@k 

Maintenant que les passages ont tous été encodés, on peut évaluer si les passages retrouvés sont pertinents. En prenant les $k$ premiers passages, on peut évaluer si on retrouve les bons passages associés aux questions. C'est un des avantages du système RAG : on peut évaluer de façon indépendante la qualité du système qui retrouve les passages pertinents et de celui qui génère les réponses. Cela permet notamment d'évaluer quels sont les points forts et points faibles du système.

Pour cela, nous allons utiliser les métriques **Precision@k** et **Recall@k** définies dans les équations suivantes. 

$$\text{Precision@k} = \frac{\text{Nombre d'éléments pertinents dans les k premiers}}{k}$$

$$\text{Recall@k} (Rappel@k) = \frac{\text{Nombre d'éléments pertinents dans les k premiers}}{\text{Nombre total d'éléments pertinents}}$$


In [19]:
def compute_recall_at_k(ground_truth, predictions):
    """
    Calcule la métrique du "Recall@k". On assume que predictions contiennent le bon nombre de passages (=k)

    Paramètres:
    ground_truth : Liste contenant tous les vrais passages associés aux questions (ex : [[1, 2, 3], [4, 5, 6]] si les passages de la question 1 sont [1, 2, 3] et
    les passages de la question 2 sont [4, 5, 6])
    predictions : Liste contenant tous les passages retrouvés pour chacune des questions formattée de la même manière que `ground_truth`

    Retourne:
    Recall moyen
    """
    # TODO
    recalls = []
    for i in range(len(ground_truth)):
        #return the size of the intersection
        nbk = len(set(ground_truth[i]) & set(predictions[i]))
        recalls.append( nbk / len(ground_truth[i]))
    return np.mean(recalls)


    # END TODO

In [54]:
def compute_precision_at_k(ground_truth, predictions):
    """
    Calcule la métrique du "Precision@k". On assume que predictions contiennent le bon nombre de passages (=k)

    Paramètres:
    ground_truth : Liste contenant tous les vrais passages associés aux questions (ex : [[1, 2, 3], [4, 5, 6]] si les passages de la question 1 sont [1, 2, 3] et
    les passages de la question 2 sont [4, 5, 6])
    predictions : Liste contenant tous les passages retrouvés pour chacune des questions formattée de la même manière que `ground_truth`

    Retourne:
    Précision moyenne
    """

    # TODO
    precisions = []
    for i in range(len(ground_truth)):
        nbk = len(set(ground_truth[i]) & set(predictions[i]))
        precisions.append(nbk / len(predictions[i]))
    return np.mean(precisions)

    # END TODO


#### 3.1.3 Résultat d'évaluation 
On implemente la fonction _'retrieve_passages'_ qui retourne les indices des $k$ passages les plus similaires pour une question. On utilise la similarité cosinus pour comparer les passages et les questions. Nous allons ensuite evaluer notre modèle de récupération en comparant notre sortie au passages pertinent du dataset de validation avec les métriques Precision@k et Recall@k.

In [20]:
import torch.nn.functional as F

In [21]:
def retrieve_passages(questions: list, passage_embed: torch.Tensor, k: int, embedding_model_tokenizer, embedding_model):
    """
    Retourne les k passages les plus pertinents pour chaque question passée en paramètre

    Paramètres:
    questions       : Les questions pour lesquelles on cherche les passages les plus pertinents
    passage_embed   : Tenseur contenant les plongements de chaque passage (n, dim)
    k               : le nombre de passages à retourner
    tokenizer       : Segmenteur du modèle de plongements
    model           : Modèle de plongements

    Retourne:
    Les indices des k passages les plus pertinents pour la question
    """
    # TODO


    all_top_k_indices = []
    batch_size = 16
    # compute cos sim with batch to not run out of memory
    for i in range(0, len(questions), batch_size):
        batch_questions = questions[i:i + batch_size]

        encoded_questions = encode_sequences(batch_questions, embedding_model_tokenizer, embedding_model)

        similarity_scores = F.cosine_similarity(
            encoded_questions.unsqueeze(1), passage_embed.unsqueeze(0), dim=2
        )

        # Best k passages
        top_k_indices = similarity_scores.topk(k, dim=1).indices
        all_top_k_indices.append(top_k_indices)

        # delete unused elements
        del encoded_questions, similarity_scores, top_k_indices
        torch.cuda.empty_cache()

    all_top_k_indices = torch.cat(all_top_k_indices, dim=0)
    return all_top_k_indices

    # END TODO


In [107]:
# TODO
best_1_passages = retrieve_passages(train["question"].to_list(), encoded_passages, 1, tokenizer, model)
best_2_passages = retrieve_passages(train["question"].to_list(), encoded_passages, 2, tokenizer, model)
best_3_passages = retrieve_passages(train["question"].to_list(), encoded_passages, 3, tokenizer, model)
best_4_passages = retrieve_passages(train["question"].to_list(), encoded_passages, 4, tokenizer, model)
# END TODO

#### 3.1.4 Graphique (3 points)
Affichons maintenant un graphique de nos résultats de la question précédente, en représentant sur l'axe des **x** la valeur de *k* et sur l'axe des **y** la précision et le rappel.  


In [109]:
# TODO
precisions_1 = compute_precision_at_k(train['text_ids'], best_1_passages.tolist())
recall_1 = compute_recall_at_k(train['text_ids'], best_1_passages.tolist())

precisions_2 = compute_precision_at_k(train['text_ids'], best_2_passages.tolist())
recall_2 = compute_recall_at_k(train['text_ids'], best_2_passages.tolist())

precisions_3 = compute_precision_at_k(train['text_ids'], best_3_passages.tolist())
recall_3 = compute_recall_at_k(train['text_ids'], best_3_passages.tolist())

precisions_4 = compute_precision_at_k(train['text_ids'], best_4_passages.tolist())
recall_4 = compute_recall_at_k(train['text_ids'], best_4_passages.tolist())
# END TODO

In [22]:
def plot_retrieval(measures,k,title,axes) :

  fig = plt.figure(figsize=(10, 5))
  ax1 = fig.add_subplot(121)

  # Tracé de la courbe avec des marqueurs et une couleur
  ax1.plot(axes, measures,
          marker='o', color='b', linestyle='-', linewidth=2, markersize=6, label=title)

  # Ajout des labels et du titre
  ax1.set_xlabel('k', fontsize=12, fontweight='bold')
  ax1.set_ylabel(title, fontsize=12, fontweight='bold')
  ax1.set_title(f'Courbe de {title} en fonction de k', fontsize=14, fontweight='bold')

  ax1.grid(True, linestyle='--', alpha=0.7)

  ax1.set_xlim(1, len(axes))
  ax1.set_ylim(0, 1)

  # Affichage
  plt.tight_layout()

In [ ]:
precisions = [precisions_1, precisions_2, precisions_3, precisions_4]
recalls = [recall_1, recall_2, recall_3, recall_4]
plot_retrieval(precisions, "k", "Precision",[1, 2, 3, 4])
plot_retrieval(recalls, "k", "Recall",[1, 2, 3, 4])

-> On cherche un équilibre entre la précision et le rappel. Une haute précision implique choisir un k plus petit mais un rapel faible. On souhaite récupérer un nombre suffisant de documents pertinents sans ajouter du bruit. Au regard des graphiques on choisit ainsi k=3. De cette manière au fait correspondre notre k au nombre moyen de documents par question étant de envion de 3.

### 3.2 RAG figé avec FAISS 

Maintenant, nous allons utiliser **FAISS** pour l'indexation des plongements des passages.  

[FAISS (*Facebook AI Similarity Search*)](https://ai.meta.com/tools/faiss/) est une bibliothèque open-source développée par Meta pour la recherche de similarité rapide sur des vecteurs denses, comme des *embeddings* de textes ou d'images. Elle est optimisée pour traiter de grands volumes de données en haute dimension et peut tirer parti des GPU pour accélérer les calculs.  

FAISS est largement utilisée dans les systèmes de recommandation et la recherche d'information à grande échelle.

#### 3.2.1 Initialisez l'indexation FAISS avec les plongements des passages 

Nous allons utiliser les plongements des passages générés à la question **3.1.1** et les indexer avec **FAISS**. Nous vérifierons que le nombre de plongements dans l'objet FAISS correspond bien au nombre de passages.  

Il est important de s'assurer que l'indexation est basée sur la **similarité cosinus** pour garantir des résultats pertinents.

In [23]:
# TODO

# déplacement sur cpu pour normaliser
encoded_passages = encoded_passages.cpu().numpy()
#Normalisation
encoded_passages = encoded_passages / np.linalg.norm(encoded_passages, axis=1, keepdims=True)

index = faiss.IndexFlatIP(encoded_passages.shape[1]) # Inner Product pour la similarité cosinus
index.add(encoded_passages)
# END TODO

In [24]:
assert index.ntotal == encoded_passages.shape[0]

print(f"Nombre de vecteurs indexés dans FAISS : {index.ntotal}")

Nombre de vecteurs indexés dans FAISS : 13314


#### 3.2.2 Récupération des passages avec FAISS 

Implémentons la fonction `_retrieve_passages_faiss_`, qui renvoie les indices des *k* passages les plus similaires à une question en utilisant une indexation avec **FAISS**.


In [25]:
def retrieve_passages_faiss(questions: list, vector_index: faiss.IndexFlatL2, k: int, embedding_model_tokenizer, embedding_model):
    """
    Retourne les k passages les plus pertinents pour chaque question passée en paramètre

    Paramètres:
    questions       : Les questions pour lesquelles on cherche les passages les plus pertinents
    vector_index    : L'objet d'indexation FAISS
    k               : le nombre de passages à retourner
    tokenizer       : Segmenteur du modèle de plongements
    model           : Modèle de plongements

    Retourne:
    Les indices des k passages les plus pertinents pour la question
    """
    # TODO
    #plongements des questions
    encoded_questions = encode_sequences(questions, embedding_model_tokenizer, embedding_model)
    encoded_questions = encoded_questions.cpu().numpy()
    encoded_questions = encoded_questions / np.linalg.norm(encoded_questions, axis=1, keepdims=True)

    #Retrieval
    dists, idx = vector_index.search(encoded_questions, k)

    return idx
    # END TODO

#### 3.2.3 Exécution de FAISS 

Nous allons maintenant exécuter la fonction `_retrieve_passages_faiss_` afin de récupérer les passages les plus pertinents pour les questions du jeu de validation, en utilisant notre valeur optimale de *k*.

In [33]:
# TODO
best_1_passages_faiss = retrieve_passages_faiss(val["question"].to_list(), index, 1, tokenizer, model)
best_2_passages_faiss = retrieve_passages_faiss(val["question"].to_list(), index, 2, tokenizer, model)
best_3_passages_faiss = retrieve_passages_faiss(val["question"].to_list(), index, 3, tokenizer, model)
best_4_passages_faiss = retrieve_passages_faiss(val["question"].to_list(), index, 4, tokenizer, model)
best_5_passages_faiss = retrieve_passages_faiss(val["question"].to_list(), index, 5, tokenizer, model)
# END TODO


#### 3.2.4 Calculons les métriques Precision@K et Rappel@K pour les passages retrouvés avec FAISS avec notre k optimal 

In [73]:
# TODO

precisions_3 = compute_precision_at_k(val['text_ids'].to_list(), best_3_passages_faiss.tolist())
recall_3 = compute_recall_at_k(val['text_ids'].to_list(), best_3_passages_faiss.tolist())


In [74]:
print(f"Precision@3: {precisions_3}")
print(f"Recall@3: {recall_3}")
# END TODO

Precision@3: 0.5593333333333333
Recall@3: 0.6031666666666666


-> Nous obtenons ainsi une précision de 0.55 et un rapel de 0.60.

Observons pour les autres k.

In [75]:
precisions_1 = compute_precision_at_k(val['text_ids'], best_1_passages_faiss.tolist())
recall_1 = compute_recall_at_k(val['text_ids'], best_1_passages_faiss.tolist())

precisions_2 = compute_precision_at_k(val['text_ids'], best_2_passages_faiss.tolist())
recall_2 = compute_recall_at_k(val['text_ids'], best_2_passages_faiss.tolist())

precisions_4 = compute_precision_at_k(val['text_ids'], best_4_passages_faiss.tolist())
recall_4 = compute_recall_at_k(val['text_ids'], best_4_passages_faiss.tolist())

precisions_5 = compute_precision_at_k(val['text_ids'], best_5_passages_faiss.tolist())
recall_5 = compute_recall_at_k(val['text_ids'], best_5_passages_faiss.tolist())

In [ ]:
precisions = [precisions_1, precisions_2, precisions_3, precisions_4, precisions_5]
recalls = [recall_1, recall_2, recall_3, recall_4, recall_5]
plot_retrieval(precisions, "k", "Precision",[1, 2, 3, 4, 5])
plot_retrieval(recalls, "k", "Recall",[1, 2, 3, 4, 5])

Remarque sur Faiss

- Exécution très rapide avec recherche par lot.
- On obtient les mêmes résultats en précision et rappel. Ce qui n'est pas étonnant, étant donné que dans les deux méthodes, nous utilisons la similarité cosinus et les même plongements.

### 3.3 Prommpt et Génération des réponses 

Nous allons maintenant générer les réponses aux questions du jeu de validation en utilisant les passages récupérés. Pour cela, nous concaténerons les passages pertinents avec la question selon un certain prompt avant de les soumettre au modèle de génération.  

In [26]:
# TODO

prompt_in_chat_format = [
    {
        "role": "system",
        "content": """Using only the information provided in the context, provide a on sentence short direct answer to the question.""",
    },
    {
        "role": "user",
        "content": """Context:
{context}
---
Here is the question you need to answer.

Question: {question}""",
    },
]

# Utilisé lors du Fine Tune pour reduire la memoire
# RAG_PROMPT_TEMPLATE = tokenizer_lora.apply_chat_template(
#     prompt_in_chat_format, tokenize=False, add_generation_prompt=True
# )

RAG_PROMPT_TEMPLATE = tokenizer_reader.apply_chat_template(
    prompt_in_chat_format, tokenize=False, add_generation_prompt=True
)

# END TODO

In [27]:
print(RAG_PROMPT_TEMPLATE)


<|system|>
Using only the information provided in the context, provide a on sentence short direct answer to the question.<|end|>
<|user|>
Context:
{context}
---
Here is the question you need to answer.

Question: {question}<|end|>
<|assistant|>



In [28]:
def add_context(passages):
  context = "\nExtracted documents:\n"
  context += "".join(
      [f"Document {str(row)}:::\n" + text["text"].iloc[row] for row in passages ]
  )
  return context


In [29]:
# TODO

def create_prompt_context(questions,passages):

  questions_context = [
      #crée le prompt pour chaque question
      RAG_PROMPT_TEMPLATE.format(
          question=question,
          context=add_context(passages[idx])
      )
      for idx, question in enumerate(questions)
  ]
  return questions_context


In [34]:
questions_prompt_1 = create_prompt_context(val["question"], best_1_passages_faiss)
questions_prompt_2 = create_prompt_context(val["question"], best_2_passages_faiss)
questions_prompt_3 = create_prompt_context(val["question"], best_3_passages_faiss)
questions_prompt_4 = create_prompt_context(val["question"], best_4_passages_faiss)

Exemple

In [48]:
questions_prompt_3[1]

'<|system|>\nUsing only the information provided in the context, provide a on sentence short direct answer to the question.<|end|>\n<|user|>\nContext:\n\nExtracted documents:\nDocument 8865:::\n  Carbapenems are structurally similar to penicillins, but with a carbon atom replacing the sulfur atom in position 1. They are biosynthesized through a series of steps involving the condensation of malonyl-CoA with glutamate-5-semialdehyde, followed by the formation of the β-lactam and saturated carbapenam core.Document 5333:::\n  Carbapenems are a class of highly effective antibiotic agents commonly used for the treatment of severe or high-risk bacterial infections. They are usually reserved for known or suspected multidrug-resistant (MDR) bacterial infections and have a broader spectrum of activity compared to most cephalosporins and penicillins.Document 2651:::\n  The growing prevalence of carbapenem-resistant Enterobacteriaceae has led to a renaissance of the use of antibiotics such as coli

### 3.4 Évaluation des réponses 
Nous allons évaluer les réponses générées par rapport aux réponses attendues sur le jeu de validation.

In [30]:
READER_LLM = pipeline(
    model=model_reader,
    tokenizer=tokenizer_reader,
    task="text-generation",
    do_sample=True,
    temperature=0.2, # Température basse pour la génération
    repetition_penalty=1.1,
    return_full_text=False,
    device= 0 if DEVICE == "cuda" else -1,
    max_new_tokens=32,
)

In [31]:
# TODO
def evaluate_rag(val,set_questions, READER):
  results = []
  #dataset ground truth
  df_true = val[["id", "answer"]]
  for set_k in set_questions :
    pred = []
    questions = set_k[0]
    k = set_k[1]
    print(f"Porcessing k = {k}")

    for question in questions :
      #génération de la réponse
      pred.append(READER(question)[0]["generated_text"])

    #dataset prediction
    df_pred = pd.DataFrame({"id": val["id"], "answer": pred})

    bleu_1 = evaluate_bleu(df_true, df_pred, 1)
    bleu_2 = evaluate_bleu(df_true, df_pred, 2)

    del df_pred
    torch.cuda.empty_cache()

    results.append([k,bleu_1, bleu_2])

  return results
# END TODO

In [35]:
set_questions = [[questions_prompt_1,1], [questions_prompt_2,2], [questions_prompt_3,3], [questions_prompt_4,4]]

In [52]:
results = evaluate_rag(val,set_questions)

Porcessing k = 1


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overla

Porcessing k = 2
Porcessing k = 3
Porcessing k = 4


In [53]:
for res in results :
  print(f"Bleu score pour k = {res[0]}")
  print(f"BLEU-1: {res[1]}")
  print(f"BLEU-2: {res[2]}")

Bleu score pour k = 1
BLEU-1: 0.3489355885101853
BLEU-2: 0.2145229330762801
Bleu score pour k = 2
BLEU-1: 0.3805855055461511
BLEU-2: 0.252961288648296
Bleu score pour k = 3
BLEU-1: 0.4205482663894729
BLEU-2: 0.2782441287195915
Bleu score pour k = 4
BLEU-1: 0.44278362113848874
BLEU-2: 0.2966836680092076


-> Bleu score pour k = 1
BLEU-1: 0.3489355885101853
BLEU-2: 0.2145229330762801

-> Bleu score pour k = 2
BLEU-1: 0.3805855055461511
BLEU-2: 0.252961288648296

-> Bleu score pour k = 3
BLEU-1: 0.4205482663894729
BLEU-2: 0.2782441287195915

-> Bleu score pour k = 4
BLEU-1: 0.44278362113848874
BLEU-2: 0.2966836680092076

Remarque:

Basé sur les scores de génération, on peut conlure que le nombre optimal est de 4. Cela montre qu'il est préférable d'ajouter plus d'informations au risque d'ajouter du bruit. Le modèle LLM pourra lui même identifier la bonne information.

### 3.5 Sortie en csv


In [54]:
best_4_passages_faiss_test = retrieve_passages_faiss(test["question"].to_list(), index, 4, tokenizer, model)
questions_prompt_4_test = create_prompt_context(test["question"], best_4_passages_faiss_test)

In [ ]:
import datetime


def create_submission(questions):

  date = datetime.datetime.now()
  date_str = str(date)[:10]
  results = []
  question_ids = test["id"]

  for question in questions :
    pred = READER_LLM(question)[0]["generated_text"]
    results.append(pred)
  #format soumissions
  df_submission = pd.DataFrame({
        'id': question_ids,
        'answer': results
    })

  df_submission.to_csv(f"submissions_{date_str}", index=False)


In [56]:
create_submission(questions_prompt_4_test)

## 4. Nouvelle methode

### 4.1. État de l'art 



Synthèse:

Le domaine de la génération augmentée par recherche (RAG), s'est vu améliorer de deux approches différentes. L'une en se contentant d'améliorer la structure initiale des RAG, les RAG avancées et l'autre en  y ajoutant des nouveaux éléments, les RAG modulaires. 

Des RAG avancées nous pouvons tirer la méthode du reranking, consitant à trier les résultats d'une première recherche dense, à l'aide de modèle Cross-encoder par exemple. Les plus récentes méthodes proposes le reranking LLM. De plus, nous pouvons noter, les techniques portant sur l'enrechissement de la requête, avec la méthode de Sub-Query. Elle consite à diviser la requête en sous requête pour contextualiser davantage la question. En effectuant plusieurs recherches de documents en parrallèle sur ces sous questions nous obtenons des résulatats enrichies.  

Pour ce qui est des RAG modulaires, nous pouvons noter les techniques adaptatives tel que Self-RAG ou encore les techniques itératives. En s'appuiant sur la technique d'augmentation de la requête avec HYDE, il est possible de modifier la réponse et la recherche jusqu'à obtenir une réponse correcte, via un JUGE LLM ou autre mesure. 

Un dernier point concerne l'affinage des modèles conséquents à l'aide de technique tel que LORA. Permettant de réduire le besoin en ressource pour fine tune le modèle générateur.

Références: Listez vos références de manière appropriée (4-5 parmi les meilleures approches)

- 1) Retrieval-Augmented Generation for Large Language Models: A Survey  -> Résume les méthodes RAG avancés et modualaire
- 2) Precise Zero-Shot Dense Retrieval without Relevance Labels -> Présente HYDE
- 3) ARAGOG: Advanced RAG Output Grading   -> Présente la combinaison HYDE + reranking LLM
- 4) Enhancing Retrieval-Augmented Large Language Models with Iterative Retrieval-Generation Synergy  -> présente une méthode itération 



### 4.2. Description de notre méthode 


Description :

Dans un premier temps, nous entraînons un modèle génératif sur le training set pour qu'il puisse répondre de manière précise et pertinente en fonction des passages extraits. Pour cela, nous utilisons la méthode LoRA, qui permet une fine-tuning sur peu de ressources. Par ailleurs, nous avons exploré l'entraînement d'un modèle d'embedding en appliquant une approche contrastive sur des exemples positifs et négatifs. Cependant, cette méthode n'a pas produit les résultats escomptés, nous conduisant à nous concentrer sur d'autres aspects de l'architecture.

Ensuite, nous mettons en œuvre deux techniques distinctes d'augmentation de requêtes en parallèle. D'une part, nous utilisons Hyde pour générer des documents hypothétiques et effectuer une recherche sur ces derniers. D'autre part, nous décomposons la requête initiale en deux ou trois sous-questions grâce à un notre modèle génératif et effectuons une recherche sur ces sous-questions. Nous avons constaté que ces deux méthodes identifient des documents complémentaires en termes de pertinence. Pour exploiter cette synergie, nous combinons les résultats des deux approches. Enfin, nous intégrons un reranking des documents à l'aide du cross-encoder ms-marco-MiniLM-L-6-v.


### 4.3. Implémentation 


##### A) Fine-tuning du modèle génératif avec LORA
Le **fine-tuning du modèle génératif avec LoRA** (*Low-Rank Adaptation*) est une méthode d’adaptation efficace qui réduit le nombre de paramètres à entraîner en insérant des matrices de faible rang dans les couches du modèle pré-entraîné. Cette approche permet de modifier le comportement du modèle sans mettre à jour l’ensemble de ses poids, rendant l'entraînement plus rapide, moins gourmand en mémoire et mieux adapté aux ressources limitées.

In [36]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# Reduis nombre de batch pour reduire le cout en memoire lors du fine-tuning
BATCH_SIZE = 16

model_lora = AutoModelForCausalLM.from_pretrained(
    READER_MODEL_NAME, 
    device_map ="auto"
)

tokenizer_lora = AutoTokenizer.from_pretrained(READER_MODEL_NAME)

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.31s/it]


In [ ]:
pip install peft

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [9]:
from peft import LoraConfig, get_peft_model

# On identifier les modules cibles 
# On va fine-tune les dernière couches
target_modules = [
    f"layers.{i}.self_attn.qkv_proj" for i in range(26, 32)
] + [
    f"layers.{i}.self_attn.o_proj" for i in range(26, 32)
]

# Configuration de LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=target_modules,
    lora_dropout=0.1,
    bias="none"
)

# Appliquer LoRA au modèle
model_lora = get_peft_model(model_lora, lora_config)


On gele les poids que nous allons pas entrainer

In [10]:
# Geler les paramètres principaux
for name, param in model_lora.named_parameters():
    if not "lora" in name:
        param.requires_grad = False


In [11]:
# Vérifier les paramètres entraînables
print("Paramètres entraînables avec LoRA :")
for name, param in model_lora.named_parameters():
    if param.requires_grad:
        print(name)

Paramètres entraînables avec LoRA :
base_model.model.model.layers.26.self_attn.o_proj.lora_A.default.weight
base_model.model.model.layers.26.self_attn.o_proj.lora_B.default.weight
base_model.model.model.layers.26.self_attn.qkv_proj.lora_A.default.weight
base_model.model.model.layers.26.self_attn.qkv_proj.lora_B.default.weight
base_model.model.model.layers.27.self_attn.o_proj.lora_A.default.weight
base_model.model.model.layers.27.self_attn.o_proj.lora_B.default.weight
base_model.model.model.layers.27.self_attn.qkv_proj.lora_A.default.weight
base_model.model.model.layers.27.self_attn.qkv_proj.lora_B.default.weight
base_model.model.model.layers.28.self_attn.o_proj.lora_A.default.weight
base_model.model.model.layers.28.self_attn.o_proj.lora_B.default.weight
base_model.model.model.layers.28.self_attn.qkv_proj.lora_A.default.weight
base_model.model.model.layers.28.self_attn.qkv_proj.lora_B.default.weight
base_model.model.model.layers.29.self_attn.o_proj.lora_A.default.weight
base_model.model

On va maintenant recupérer nos données d'entrainement en utilisant le prompt précédent

In [16]:
# Convertie les indices en données numerique et pas string
train_textids = train["text_ids"].apply(lambda x: [int(i) for i in x.replace("[", "").replace("]", "").split(" ") if i.isdigit()])
# Crée une colonne avec les prompt de chaque question
trainset_prompt = create_prompt_context(train["question"], train_textids)
train["prompt"] = trainset_prompt

Formate nos données d'entrainement pour qu'elles soient utilisable our un model Causal

In [17]:
def format_data(row):
    input_text = row['prompt']
    target_text = row['answer']
    full_text = input_text + target_text
    return pd.Series({"input_text": input_text, "target_text": target_text, "full_text": full_text})

formatted_train = train.apply(format_data, axis=1)

In [18]:
# Initialisation des compteurs
max_tokens = 0

# Parcourir chaque ligne pour analyser les tokens
for idx, row in formatted_train.iterrows():
    tokenized = tokenizer_lora(
        row["full_text"],
        truncation=False,  # Pas de troncature pour capturer toute la longueur
        return_tensors="pt"
    )
    num_tokens = len(tokenized["input_ids"][0])
    
    # Mise à jour du maximum de tokens
    if num_tokens > max_tokens:
        max_tokens = num_tokens

# Affichage du résultat
print(f"Nombre maximum de tokens : {max_tokens}")


Nombre maximum de tokens : 598


In [19]:
def tokenize_data(row):
    # Tokeniser les questions + réponses ensemble pour le modèle causal
    tokens = tokenizer_lora(
        row["full_text"],  # Texte complet (prompt + réponse)
        max_length=600,  # Limite selon le max de token pour éviter les séquences trop longues
        truncation=True,
        padding="max_length"
    )
    tokens["labels"] = tokens["input_ids"].copy()  # Les labels sont identiques aux entrées
    return tokens

# Appliquer la fonction ligne par ligne
tokenized_train = formatted_train.apply(lambda row: tokenize_data(row), axis=1)

In [20]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_train,
    shuffle=True,
    batch_size=BATCH_SIZE,
    collate_fn=lambda x: {k: torch.tensor([d[k] for d in x]) for k in x[0]}
)


On passe à l'entrainement

In [21]:
from transformers import AdamW

# On inclut uniquement les paramètres non gelés
optimizer = AdamW(
    [param for param in model_lora.parameters() if param.requires_grad],
    lr=5e-5
)


/teamspace/studios/this_studio/.venv/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [22]:
from transformers import get_scheduler
# Scheduler de taux d'apprentissage (learning rate scheduler) 
# Utile pour ajuster dynamiquement le taux d'apprentissage pendant l'entraînement

num_training_steps = len(train_dataloader) * 3  # 3 époques
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)


In [23]:
from tqdm import tqdm

model_lora.train()  # Passer le modèle en mode entraînement

progress_bar = tqdm(range(num_training_steps))
torch.cuda.empty_cache()  # Libérer la mémoire inutilisée avant l'entraînement

accumulation_steps = 2  # Divise chaque batch en 2 sous-batches
running_loss = 0.0  # Variable pour suivre la perte cumulée

for epoch in range(3):  # Boucle des époques
    print(f"Époque {epoch + 1}/{3}")
    for i, batch in enumerate(train_dataloader):
        batch = {k: v.to(DEVICE) for k, v in batch.items()}
        outputs = model_lora(**batch)
        loss = outputs.loss
        loss = loss / accumulation_steps  # Diviser la perte pour l'accumulation
        loss.backward()

        running_loss += loss.item()  # Ajouter la perte actuelle à la somme

        if (i + 1) % accumulation_steps == 0 or (i + 1) == len(train_dataloader):  # Après accumulation_steps
            optimizer.step()
            optimizer.zero_grad()

            # Afficher la perte moyenne
            avg_loss = running_loss / accumulation_steps
            print(f"Batch {i + 1}/{len(train_dataloader)} - Loss moyenne : {avg_loss:.4f}")
            running_loss = 0.0  # Réinitialiser la perte cumulée

        progress_bar.update(1)

# Sauvegarde du modèle après l'entraînement
model_lora.save_pretrained("fine_tuned_model")
tokenizer_lora.save_pretrained("fine_tuned_model")


  0%|          | 0/657 [00:00<?, ?it/s]

Époque 1/3


  0%|          | 2/657 [00:28<2:13:55, 12.27s/it]

Batch 2/219 - Loss moyenne : 4.5233


  1%|          | 4/657 [00:32<55:01,  5.06s/it]  

...

100%|█████████▉| 656/657 [18:06<00:01,  1.60s/it]

Batch 218/219 - Loss moyenne : 0.2738


100%|██████████| 657/657 [18:07<00:00,  1.31s/it]

Batch 219/219 - Loss moyenne : 0.1269


('fine_tuned_model/tokenizer_config.json',
 'fine_tuned_model/special_tokens_map.json',
 'fine_tuned_model/tokenizer.json')

In [24]:
print(loss)

tensor(0.2538, device='cuda:0', grad_fn=<DivBackward0>)


Chargement du modèle fine tuned

In [37]:
from peft import PeftModel

model_lora = PeftModel.from_pretrained(model_lora, "fine_tuned_model")
tokenizer_lora = AutoTokenizer.from_pretrained("fine_tuned_model")

In [38]:
LORA_LLM = pipeline(
    model=model_lora,
    tokenizer=tokenizer_lora,
    task="text-generation",
    do_sample=True,
    temperature=0.2, # Température basse pour la génération
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=32,
)

The model 'PeftModel' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'Mega

##### B) Modèle de reranking 

Nous avons également utilisé du reranking. Nous avons utilisé le cross-encoder MiniLM.

In [39]:
# Chargement du tokenizer et du modèle LLM open source
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer_rerank =  AutoTokenizer.from_pretrained("cross-encoder/ms-marco-MiniLM-L-6-v2")
model_rerank = AutoModelForSequenceClassification.from_pretrained("cross-encoder/ms-marco-MiniLM-L-6-v2")
model_rerank.eval()  # Passer en mode évaluation

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 384, padding_idx=0)
      (position_embeddings): Embedding(512, 384)
      (token_type_embeddings): Embedding(2, 384)
      (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-5): 6 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (LayerNorm): LayerNorm((384,), eps=1e-1

In [40]:
import torch
import numpy as np

def rerank_with_minilm(question, passage_indices, corpus, model, tokenizer, top_k=4):
    
    passages = [corpus[idx] for idx in passage_indices]
    # Construire les paires (question, passage)
    pairs = [(question, passage) for passage in passages]
    
    # Tokenisation des paires
    tokenized_inputs = tokenizer(
        [pair[0] for pair in pairs],
        [pair[1] for pair in pairs],
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    ).to(model.device)
    
    # Générer les scores par le modèle
    with torch.no_grad():
        outputs = model(**tokenized_inputs)
        scores = outputs.logits.squeeze(-1).cpu().numpy()  # Scores de pertinence

    # Associer les scores avec les indices globaux des passages
    ranked_indices = sorted(
        zip(passage_indices, scores), key=lambda x: x[1], reverse=True
    )
    
    # Retourner les indices globaux des top_k passages
    return [index for index, score in ranked_indices[:top_k]]


##### C) Méthode de division de la requête : Sub-queries

Création des sous-questions

In [41]:
#llm_subquerie =  pipeline("text-generation", model=model_reader,tokenizer=tokenizer_reader)

In [42]:
import re

def generate_subqueries(query, llm, num_queries=3):
    
    subqueries = []

    if len(query) < 40:
        num_queries -= 1
    
    prompt = f"Your task is only to divide this question [{query}] in maximum {num_queries} differents shorts subquestions names n1,n2 and n3."
    response = llm(prompt, max_new_tokens=100)
    
    subqueries= response[0]["generated_text"].strip()
    subq = re.findall(r"n\d+:\s*(.*?)(?=\n|$)", subqueries)
    if subq ==[]:
        subq = subqueries
    return subq[:num_queries]

In [43]:
import json

#Val
#sub_val = []

# for question in val["question"]:
#     subqueries = generate_subqueries(question, llm_subquerie, num_queries=3)
#     sub_val.append(subqueries)

# # sauvegarde
# with open('sub_val.json', 'w', encoding='utf-8') as f:
#     json.dump(sub_val, f, ensure_ascii=False, indent=4)

#Test 
#sub_test = []

# for question in test["question"]:
#     subqueries = generate_subqueries(question, llm_subquerie, num_queries=3)
#     sub_test.append(subqueries)

# # sauvegarde
# with open('sub_test.json', 'w', encoding='utf-8') as f:
#     json.dump(sub_test, f, ensure_ascii=False, indent=4)


Chargement des sous-questions sauvegardés

In [44]:
with open('sub_val.json', 'r', encoding='utf-8') as f:
    sub_val = json.load(f)

    
with open('sub_test.json', 'r', encoding='utf-8') as f:
    sub_test = json.load(f)

Exemple

In [45]:
sub_val[1]

['What is the core structure of carbapenems?',
 'How does the side chain of carbapenems differ from penicillins?',
 'What functional groups are present in carbapenems that are not in penicillins?']

Recherche avec les sous questions

In [46]:
from collections import Counter
from itertools import chain

def select_best_documents(results,k):
    
    # occurrences de chaque document
    doc_counts = Counter(results)
    
    # vote majoritaire
    sorted_docs = sorted(doc_counts.items(), key=lambda x: (-x[1], x[0]))  # tri par fréquence, puis par indice
    top_docs = [doc[0] for doc in sorted_docs[:k]]

    return top_docs


In [47]:
def retrieve_passages_subq(questions: list, vector_index: faiss.IndexFlatIP, k: int,
                           embedding_model_tokenizer, embedding_model):
   
    all_indices = []

    for subqueries in questions:
        #subqueries = generate_subqueries(question, llm_subquerie, num_queries = 3)
        results = []
        for subquery in subqueries:
            
            encoded_sub_query = encode_sequences([subquery], embedding_model_tokenizer, embedding_model)
            encoded_sub_query = encoded_sub_query.cpu().numpy()
        
            
            dists, idx = vector_index.search(encoded_sub_query, k)
            
            results.append(idx[0].tolist())
        
        results = list(chain.from_iterable(results))
        
        best_indices = select_best_documents(results,k)
        
        all_indices.append(best_indices)

    return all_indices

##### D) Augmentation de la recherche avec Hyde 

In [48]:
prompt_in_chat_format_rewrite = [
    {
        "role": "system",
        "content": """Using only the information provided in the context, answer the question.""",
    },
    {
        "role": "user",
        "content": """Context:
{context}
---
Here is the question you need to develop.

Original Question: {question}""",
    },
]

RAG_PROMPT_TEMPLATE_REWRITE = tokenizer_reader.apply_chat_template(
    prompt_in_chat_format_rewrite, tokenize=False, add_generation_prompt=True
)

In [49]:
def retrieve_passages_hyde(questions: list, vector_index: faiss.IndexFlatIP, k: int,
                           embedding_model_tokenizer, embedding_model, corpus):
    
    all_indices = []

    for question in questions:
        
        # encodage de la requête
        encoded_query = encode_sequences([question], embedding_model_tokenizer, embedding_model)
        encoded_query = encoded_query.cpu().numpy()
        encoded_query /= np.linalg.norm(encoded_query, axis=1, keepdims=True)

        # Recherche initiale dans FAISS
        dists, indices = vector_index.search(encoded_query, k)
        context = " ".join([corpus[i] for i in indices[0]] if isinstance(corpus, list) else corpus.iloc[indices[0]].tolist())
        
        # réécriture de la question avec HYDE
        question_rw = RAG_PROMPT_TEMPLATE_REWRITE.format(
            question=question,
            context=context
        )
        pred_rw = READER_LLM(question_rw)[0]["generated_text"]
        
        question_final =  pred_rw.strip() + question 
        
        # encodage du document hypothétique
        encoded_hypothetical = encode_sequences([question_final], embedding_model_tokenizer, embedding_model)
        encoded_hypothetical = encoded_hypothetical.cpu().numpy()
        

        # recherche finale dans l'index FAISS
        dists, idx = vector_index.search(encoded_hypothetical, k)
        
        all_indices.append(idx.tolist()[0])

    return all_indices


##### E) Combinaison des documents de Hyde et Sub-queries

In [50]:
from collections import defaultdict

# Combine deux listes en préservant l'ordre et en favorisant les résultats d'une liste via des pondérations.
def combine_lists_preserve_order_with_weights(list1, list2, k, weight1=1.0, weight2=1.0):
    
    scores = defaultdict(float)
    
    for rank, idx in enumerate(list1):
        scores[idx] += weight1 * (1 / (rank + 1))  
    
    for rank, idx in enumerate(list2):
        scores[idx] += weight2 * (1 / (rank + 1))  
    
    # tri des indices par score décroissant
    sorted_indices = sorted(scores.items(), key=lambda x: -x[1])
    
    top_indices = [idx for idx, score in sorted_indices[:k]]
    
    return top_indices


##### F) Implémentation complète 

In [51]:
def retrieve_passages_combined(questions,subs, vector_index, k, embedding_model_tokenizer, embedding_model, corpus,
                               gen_tokenizer, gen_model, model_rerank, tokenizer_rerank,n=10):
    all_indices = []

    for question,subqueries in zip(questions,subs):
        
        # Méthode HYDE
        hyde_indices = retrieve_passages_hyde([question], vector_index, n, embedding_model_tokenizer, embedding_model, corpus)
        
        # Méthode Sub-Q
        subq_indices = retrieve_passages_subq([subqueries], vector_index, n, embedding_model_tokenizer, embedding_model)
        
        # Combiner les résultats
        combined_results = combine_lists_preserve_order_with_weights(hyde_indices[0], subq_indices[0], n,weight1=1.5, weight2=1.0)
        
        #Reranking -> top k
        rerank_indices = rerank_with_minilm(question, combined_results, corpus, model_rerank, tokenizer_rerank, top_k=4)

        
        all_indices.append(rerank_indices)

    return all_indices


### 4.4. Évaluation 


Retrieve

In [52]:
best_4_ids = retrieve_passages_combined(val["question"],sub_val, index, 4, tokenizer, model, text["text"],tokenizer_reader, model_reader, model_rerank, tokenizer_rerank)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [55]:
precisions_4 = compute_precision_at_k(val['text_ids'], best_4_ids)
recall_4 = compute_recall_at_k(val['text_ids'], best_4_ids)
print(f"Precision@4: {precisions_4}")
print(f"Recall@4: {recall_4}")

Precision@4: 0.498
Recall@4: 0.7116666666666667


Prompt

In [56]:
eval_promt= create_prompt_context(val["question"], best_4_ids)

In [57]:
set_questions = [ [eval_promt,4]]

BLEU-2

In [58]:
res = evaluate_rag(val,set_questions, LORA_LLM)

Porcessing k = 4


/teamspace/studios/this_studio/.venv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/teamspace/studios/this_studio/.venv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/teamspace/studios/this_studio/.venv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.


In [61]:
print(f"Bleu-2 = {res[0][2]}")

Bleu-2 = 0.29649959055100494


### 4.5. Analyse 
#### 4.5.1. Avantages/limites/Erreurs types 



Notre architecture pour la recherche n'a pas réussi à surpasser significativement le modèle baseline.

Le prompt du modèle génératif semble pertinent en augmentant significativement les performances au départ de nos recherches. Le fine-tuning a présenté là aussi une légère amélioration. Il nous semble claire que le goulot d'étranglement réside dans le score de précision qui est très faible. En mettant un k à 4, nous augmentons le rappel mais ajoutant beaucoup de bruits à cause d'une précision faible.  

Bien que nous ayons implémenté des techniques visant à enrichir la recherche en obtenant des documents pertinents différents de ceux identifiés par la baseline, nous n'avons pas su combiner ces approches de manière optimale pour améliorer le score de précision.  


#### 4.5.2. Améliorations potentielles




Pour réduire l'impacte d'une faible précision, il peut être pertinent d'implémenter un juge entrainé sur le training set pour déterminer si la réponse et suffisante. Et ainsi implémenter une itération sur la recherche et génération. Nous avons essayer d'utiliser le modèle génératifs sans affinage comme juge mais il n'était pas performant.

Une piste d'amélioration pourrait être un entraînement end-to-end, intégrant à la fois le modèle générant les documents hypothétiques et le modèle d'embedding, afin d'orienter de manière optimale la génération et l'embedding. 

